## horseIDとjockeyIDの取得

In [354]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from tqdm import tqdm

In [355]:

df = pd.read_pickle('inner_df.pickle')
race_id_list = df.index.unique()
type(race_id_list[0])
# race_id_list = ["202101010101","202101010102"]

str

In [356]:
test = {}
def scrape_race_results(race_id_lists,par_race_results={}):
  race_results = par_race_results
  for _race_id in tqdm(race_id_lists):
    # if _race_id in race_result.keys():
    #   continue
    time.sleep(1)
    try:
      # スクレイピング
      url = "https://race.netkeiba.com/race/result.html?race_id=" + _race_id
      html = requests.get(url)
      html.encoding= "EUC_JP"
      soup = BeautifulSoup(html.text, "html.parser")
      
      # horse_id
      horse_id_list = []      
      horse_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/horse/")})
      for a in horse_a_list:
        horse_id = re.findall(r'\d+',a["href"])
        horse_id_list.append(horse_id[0])
      Series1 = pd.Series(data = horse_id_list,dtype=str,name="horse_id")
      # jockey_id
      jockey_id_list = []
      jockey_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/jockey/")})
      for u in jockey_a_list:
        jockey_id = re.findall(r'\d+',u["href"])
        jockey_id_list.append(jockey_id[0])
      Series2 = pd.Series(data = jockey_id_list,dtype=str,name="jockey_id")
      
      results = pd.concat([Series1, Series2], axis=1)
      test[_race_id] = results
      
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
    except TimeoutError:
      break
  return test

In [357]:
results_new = scrape_race_results(race_id_list)
results_new


100%|██████████| 36/36 [00:49<00:00,  1.37s/it]


{'202101010101':       horse_id jockey_id
 0   2018105875     01181
 1   2018105258     01140
 2   2018101134     01096
 3   2018102791     01157
 4   2018101539     01015
 5   2018100570     00666
 6   2018102192     01170
 7   2018102626     01095
 8   2018103582     01180
 9   2018101518     01173
 10  2018106100     01192
 11  2018101447     01091
 12  2018109106     01112
 13  2018106896     01156
 14  2018104780     01144
 15  2018100737     05203,
 '202101010102':       horse_id jockey_id
 0   2018105205     00666
 1   2018100974     01161
 2   2018104870     01182
 3   2018106453     01096
 4   2018100926     01170
 5   2018102052     01157
 6   2018103196     01025
 7   2018101898     01187
 8   2018102542     01192
 9   2018103594     01124
 10  2018106609     01185
 11  2018100435     01166
 12  2018105770     01172
 13  2018100706     01140,
 '202101010103':       horse_id jockey_id
 0   2018100403     01157
 1   2018105166     01182
 2   2018102691     01091
 3   201810070

In [358]:
for key in results_new.keys():
  # print(key)
  # print(results_new[key].index)
  results_new[key].index = [key] * len(results_new[key])
results = pd.concat([results_new[key] for key in results_new.keys()],sort=False)

In [364]:
print(len(results))
print(len(df))

468
468


In [367]:
pd.set_option('display.max_rows',None)

df_add_id = df.merge(results,left_index=True,right_index=True,how="inner")

In [369]:
df_add_id.to_pickle("df_add_id.pickle")